Read the Postgres Database and Generate Table 2 and Table 3
-----------------------------------------------------------
This notebook does not modify the database.

In [1]:
val userhome = System.getProperty("user.home")
val jniPath = s"${userhome}/software/z3/build"
val newPath = Array(jniPath) ++  System.getProperty("java.library.path").split(":")
System.setProperty("java.library.path",newPath.distinct.mkString(":"))
val sysPathsField = classOf[ClassLoader].getDeclaredField("sys_paths");
sysPathsField.setAccessible(true);
sysPathsField.set(null, null);
val jarpath = s"/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
interp.load.cp(os.Path(jarpath))

import $ivy.`org.plotly-scala:plotly-almond_2.13:0.8.2`
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import $ivy.`com.github.nscala-time::nscala-time:2.32.0`

import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import plotly._, element._, layout._, Plotly._
import ujson.Value
import sys.process._

import $ivy.`com.lihaoyi::scalatags:0.12.0`

import jupyter.Displayer, jupyter.Displayers
import scala.collection.JavaConverters._
import scala.collection.mutable
import com.github.nscala_time.time.Imports._
import org.joda.time.Period

userhome: String = "/root"
jniPath: String = "/root/software/z3/build"
newPath: Array[String] = Array(
  "/root/software/z3/build",
  "/usr/lib/",
  "/usr/java/packages/lib",
  "/usr/lib64",
  "/lib64",
  "/lib",
  "/usr/lib"
)
res0_3: String = "/usr/lib/:/usr/java/packages/lib:/usr/lib64:/lib64:/lib:/usr/lib"
sysPathsField: java.lang.reflect.Field = private static java.lang.String[] java.lang.ClassLoader.sys_paths
jarpath: String = "/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                          

import $ivy.$                                         

import $ivy.$                                           


import $ivy.$                             
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`


import plotly._, element._, layout._, Plotly._

import ujson.Value

import sys.process._


import $ivy.$                              


import jupyter.Displayer, jupyter.Displayers

import scala.collection.JavaConverter

In [2]:
import better.files._

import scala.util.Random
import edu.colorado.plv.bounder.{Driver,ExpTag,Action,RunConfig,PickleSpec}
import edu.colorado.plv.bounder.lifestate.{SpecSpace,ViewSpec,SpecSignatures}
import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,Reachable,ReceiverNonNull, DisallowedCallin}
import upickle.default.read
import upickle.default.write
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
import edu.colorado.plv.bounder.ExperimentsDb
import edu.colorado.plv.bounder.BounderUtil
//import scala.concurrent.duration._
import scala.language.postfixOps
import slick.driver.H2Driver.api._
import slick.jdbc.GetResult
import slick.jdbc.SQLActionBuilder
import scala.concurrent.Await
import almond.interpreter.api.DisplayData
import edu.colorado.plv.bounder.ir.Messages


Driver.setZ3Path(s"${userhome}/software/z3/build")

// var android_home_possible = List(s"${userhome}/Library/Android/sdk", s"${userhome}/Android/Sdk")
// var android_home = android_home_possible.find(p => File(p).exists()).get             
// BounderUtil.setEnv(Map("DYLD_LIBRARY_PATH" -> s"${userhome}/software/z3/build","ANDROID_HOME" -> android_home,"HOME" -> userhome))

System.setProperty("user.dir", s"${System.getProperty("user.home")}/Documents/source/bounder/notebooks/ossExp/SpecGen");
val expDir = File("/home/notebooks")

java.library.path set to: /root/software/z3/build:/usr/lib/:/usr/java/packages/lib:/usr/lib64:/lib64:/lib:/usr/lib


import better.files._


import scala.util.Random

import edu.colorado.plv.bounder.{Driver,ExpTag,Action,RunConfig,PickleSpec}

import edu.colorado.plv.bounder.lifestate.{SpecSpace,ViewSpec,SpecSignatures}

import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,Reachable,ReceiverNonNull, DisallowedCallin}

import upickle.default.read

import upickle.default.write

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}

import edu.colorado.plv.bounder.ExperimentsDb

import edu.colorado.plv.bounder.BounderUtil
//import scala.concurrent.duration._

import scala.language.postfixOps

import slick.driver.H2Driver.api._

import slick.jdbc.GetResult

import slick.jdbc.SQLActionBuilder

import scala.concurrent.Await

import almond.interpreter.api.DisplayData

import edu.colorado.plv.bounder.ir.Messages



res1_18: String = "/home/notebooks"
expDir: File = /home/notebooks

In [3]:
val db = new ExperimentsDb(Some(jarpath))
case class Count(n:Int)
case class StrRes(s:String)
implicit val getCountResult = GetResult(r => Count(r.<<))
implicit val getStrResult = GetResult(r => StrRes(r.<<))

//implement in a class so any sequence gets converted to a table that can be displayed in jupyter
trait TableAble {
  def headers:List[String]   
  def values:List[Any]
}
case class Table(v:Seq[TableAble]){

}
Displayers.register(classOf[Table], (t: Table) => {
  import scalatags.Text.all._
  Map(
    "text/html" -> {
      table(cls:="table")(
        tr(t.v.head.headers.map(v => th(v))),
        for (row <- t.v) yield tr(
          row.values.map{
              case v:String => td(v)
              case v:Int => td(v)
          }
        )
      ).render
    }
  ).asJava
})

Initializing database


db: ExperimentsDb = edu.colorado.plv.bounder.ExperimentsDb@431065e8
defined class Count
defined class StrRes
getCountResult: AnyRef with GetResult[Count] = <function1>
getStrResult: AnyRef with GetResult[StrRes] = <function1>
defined trait TableAble
defined class Table

In [4]:
//TODO: this is super slow and should probably be fixed at some point
object AppStats{
    val baseDataDir = File("/home/historia_generalizability/")
    val appList = File("benchmark-list.txt").contentAsString.split('\n').map{v => 
        val path = v.split(':')(0).split("historia_generalizability/")(1)
        val filter = v.split(':')(1)
        (path,filter)
    }
    val appFileList = appList.map{case (apk,filter) => (baseDataDir / apk, filter)}
    def getLoc(dir:File):Int = {
        val out = BounderUtil.runCmdStdout(s"cloc --json ${dir}")
        val jsonLoc = ujson.read(out)
        val langs = Set("Java","Kotlin","Scala")
        if(langs.forall(l => !jsonLoc.obj.contains(l))){
            println(s"exn: ${dir}\n${jsonLoc}")
            ???
        }
        langs.foldLeft(0){
            case (acc, v) => 
                if(jsonLoc.obj.contains(v))
                    acc + jsonLoc(v)("code").num.toInt
                else{
                    acc
                }
        }
    }
    def getCINameCount(dir:File, filt:String):Int = {
        val filtLike = s"'%$filt%'"
        // count distinct names of callins (e.g. Button.setTitle(String) , TextView.setTitle(String), and DialogBuilder.setTitle(Int) would be 2 callins)
        val out = BounderUtil.runCmdStdout(s"""sqlite3 ${dir}/out.db "select count(distinct tgt_name) from call_edges where is_callin>0 and src_class like ${filtLike}" """)
        out.trim.toInt
    }
    def getCBNameCount(dir:File, filt:String):Int = {
        val filtLike = s"'%$filt%'"
        val out = BounderUtil.runCmdStdout(s"""sqlite3 ${dir}/out.db "select count(distinct name) from methods where is_callback and declaring_class like ${filtLike}" """)
        out.trim.toInt
    }
    def getCBSynt(dir:File, filt:String):Int = { // syntactic methods considered a callback
        val filtLike = s"'%$filt%'"
        val out = BounderUtil.runCmdStdout(s"""sqlite3 ${dir}/out.db "select count(*) from methods where is_callback and declaring_class like ${filtLike}" """)
        out.trim.toInt
    }
    def getAppMethods(dir:File, filt:String):Int = {
        val filtLike = s"'%$filt%'"
        val out = BounderUtil.runCmdStdout(s"""sqlite3 ${dir}/out.db "select count(*) from methods where declaring_class like ${filtLike}" """)
        out.trim.toInt
    }
    
    def withSrc[T](apkPath:File, f:File => T):T = {
        import better.files.Dsl._
        if(apkPath.toString.contains("org.connectbot")){
            f(File("/home/historia_generalizability/oss/org.connectbot/"))
        }else if(apkPath.toString.contains("de.danoeh.antennapod")){
            f(File("/home/historia_generalizability/oss/de.danoeh.antennapod/"))
        }else{
            var out:Option[T] = None
            File.usingTemporaryDirectory(){ dir => 
                //println(dir.toString)
                val tarFile = dir / "src.tar.gz"
                val baseDir = apkPath.parent.parent / "src"
                for{
                    f <- baseDir.glob("*.tar.gz")
                }{f.copyTo(tarFile)}
                // ungzip(dir / "src.tar.gz")(dir / "src")
                BounderUtil.runCmdStdout(s"""bash -c "cd ${dir} ; tar -xvzf src.tar.gz" """)
                
                // dir.glob("**").foreach{println}
                out = Some(f(dir))
            }
            out.get
        }
    }
    def withInfoDB[T](apkPath:File, f:File => T):T = {
        import better.files.Dsl._
        println(apkPath)
        var res:Option[T] = None
        File.usingTemporaryDirectory(){ dir => 
            val config = RunConfig(apkPath=apkPath.toString, outFolder=Some(dir.toString))
            val act = Action(mode=Driver.Info, config=config, baseDirOut = Some(dir.toString), baseDirApk = Some(dir.toString))
            // Driver.info(config,"","")
            act.runCmdFork(jarpath)
            
            res = Some(f(dir))
        }
        res.get
    }
    case class Stats(apk:File, filter:String) extends TableAble{
       val loc = withSrc(apk, getLoc)
       // val (callinNameCount, callbackNameCount, callbackSynt, appMethods) = withInfoDB(apk, (f:File) => {
       //    (getCINameCount(f, filter), getCBNameCount(f,filter), getCBSynt(f,filter), getAppMethods(f,filter)) 
       // })
       def headers:List[String] = "apk"::"lines of code"::Nil//, "callin name count", "callback name count", "callback synt", "app methods")
       def values:List[Any] = apk.toString::loc::Nil //, callinNameCount, callbackNameCount, callbackSynt, appMethods)
    }
}
//AppStats.withInfoDB(AppStats.appFileList.head._1, f => AppStats.getCINameCount(f, AppStats.appFileList.head._2))
// println(s"callin: ${AppStats.getCINameCount(File("/Users/shawnmeier/Desktop/specsRun/de.danoeh.antennapod"), "de.danoeh.antennapod")}")
// println(s"callback: ${AppStats.getCBNameCount(File("/Users/shawnmeier/Desktop/specsRun/de.danoeh.antennapod"), "de.danoeh.antennapod")}")
// println(s"callback synt: ${AppStats.getCBSynt(File("/Users/shawnmeier/Desktop/specsRun/de.danoeh.antennapod"), "de.danoeh.antennapod")}")
// println(s"app methods: ${AppStats.getAppMethods(File("/Users/shawnmeier/Desktop/specsRun/de.danoeh.antennapod"), "de.danoeh.antennapod")}")
//TODO: uncomment stats to compute whole thing
//val stats = AppStats.appFileList.map{case (f,filter) => AppStats.Stats(f, filter)}
// object WriteLoc{
//     val locString = stats.map{s => s"${s.apk},${s.filter},${s.loc}"}.mkString("\n")
//     def write() = {
//         val f = File("locCount.csv")
//         f.overwrite(locString)
//     }
// }
// WriteLoc.write()

defined object AppStats

In [5]:
case class JoinedResultRow(tag:String, summary:String, loc:List[String], qry:String, config:String, apk:String) extends TableAble{
    override def toString = s"JoinedResultRow(summary = ${summary}, apk = ${apk}"
    def getSummary:BounderUtil.ResultSummary = read[BounderUtil.ResultSummary](summary)
    val getConfig:RunConfig = read[RunConfig](config)
    val getInitQuery = read[InitialQuery](qry)
    val getInitialQuery = getInitQuery
    assert(getConfig.initialQuery.exists{q => q == getInitQuery})
    def headers = List("tag", "summary", "qry", "apk")
    def values = List (tag  ,  summary , qry  , apk)
    def merge(other:JoinedResultRow):JoinedResultRow = {
        assert(tag == other.tag)
        assert(qry == other.qry)
        assert(config == other.config)
        assert(apk == other.apk)
        JoinedResultRow(tag, write[BounderUtil.ResultSummary](Driver.reduceResults(getSummary, other.getSummary)), loc ++ other.loc,qry, config, apk)
    }
}
object JoinedResultRow{
    def fromConfig(c:RunConfig):JoinedResultRow = {
        assert(c.initialQuery.size == 1)
        JoinedResultRow(tag=c.tag.heuristicType,summary = "unknown", Nil, write[InitialQuery](c.initialQuery.head), write[RunConfig](c), "")
    }
}

case class SummaryRow(witnessed:Set[JoinedResultRow] = Set.empty, unreachable:Set[JoinedResultRow] = Set.empty, 
                      timeout:Set[JoinedResultRow] = Set.empty, err:Set[JoinedResultRow] = Set.empty) extends TableAble{
    def headers =  List("witnessed", "unreachable", "timeout", "err")
    def values = List(witnessed.size, unreachable.size, timeout.size, err.size)
    def add(row:JoinedResultRow):SummaryRow = {
        row.getSummary match{
            case BounderUtil.Timeout => this.copy(timeout = timeout + row)
            case BounderUtil.Witnessed => this.copy(witnessed = witnessed + row)
            case BounderUtil.Unreachable => this.copy(unreachable = unreachable + row)
            case BounderUtil.Interrupted(_) => this.copy(err = err + row)
        }
    }
}
case class ApkSummaryRow(apk:String, row:SummaryRow = SummaryRow()) extends TableAble{
    def headers = "apk"::row.headers
    def values = apk::row.values
    def add(newRow:JoinedResultRow):ApkSummaryRow = {
        assert(newRow.apk == apk)
        this.copy(row = row.add(newRow))
    }
}




implicit val getRowResult = GetResult(r => JoinedResultRow(r.<<, r.<<, List(r.<<), r.<<, r.<<, r.<<))
object ReadResults{
    import scala.concurrent.duration.Duration
//where cast(results.result::json->'summary' as varchar) like '%Witnessed%' or cast(results.result::json->'summary' as varchar) like '%timeout%'
    //TODO:==== figure out if this query is counting things correctly
    def selectResults() = {
        val q = sql"""select results.jobtag, cast(results.result::json->'summary' as varchar), results.loc, results.qry, jobs.config as cfg, 
        cast(jobs.config::json->'apkPath' as varchar) as benchmark from results inner join jobs on jobs.id=results.jobid"""
        val res = Await.result(db.db.run(q.as[JoinedResultRow]), Duration(30,"seconds"))
        res.filter(res => read[ExpTag](res.tag).specRefinement == "")
            .groupBy(res => res.getConfig).map{ // each query can have multiple locations, we just merge them
            case (_, rows) => rows.reduce((a:JoinedResultRow,b:JoinedResultRow) => a.merge(b))
        }.toSet
    }
    private val allResults = selectResults()

    val rand = new scala.util.Random
    
    // Top model
    val allTopResults = allResults
    val topResultsByQuery = allTopResults
    val topByApp:Seq[ApkSummaryRow] = topResultsByQuery.foldLeft(Map[String,ApkSummaryRow]()){
        case (acc,joinedRow) => 
            acc + (joinedRow.apk -> acc.getOrElse(joinedRow.apk, ApkSummaryRow(joinedRow.apk)).add(joinedRow))    
    }.values.toSeq.sortBy(_.apk)
    val appList = topByApp.map{v => v.apk}
    val topByResult = Seq(topResultsByQuery.foldLeft(SummaryRow()){
        case (acc, joinedRow) => acc.add(joinedRow)
    })
    def summaryForApkTop(app:String) = {
        val res = topByApp.filter(row => row.apk.contains(app))
        assert(res.size == 1, s"Failure, wrong number of results ${res.size}")
        res.head
    }
    def sampleWitForTop(app:String) = {
        val summary = summaryForApkTop(app)
        val witnesses = summary.row.witnessed.toList
        if(witnesses.isEmpty)
            None
        else{
            val max = witnesses.size
            Some(witnesses(rand.nextInt(max)))
        }
    }

    // ***summarize results by category***
    case class TagCount(tag:String, count:Int) extends TableAble{
        def headers = "tag"::"count"::Nil
        def values = tag::count::Nil
    }
    
    case class CategorySummaryRow(tag:String, total:Set[JoinedResultRow], witnessed:Set[JoinedResultRow], 
                              unreachable:Set[JoinedResultRow], timeout:Set[JoinedResultRow]) extends TableAble{
        val (headersV, valuesV) = List(
                ("tag", tag),
                ("total", total.size),
                ("apps", appsFor()),
                ("witnessed", witnessed.size),
                ("unreachable", unreachable.size),
            ).unzip
        def headers = "tag"::"total"::"apps"::"witnessed"::"unreachable"::"timeout"::Nil
        def appsFor():Int = {
            total.map{row => row.getConfig.apkPath}.toSet.size
        }
        def getApps = total.map{row => row.getConfig.apkPath}.toSet
        def values = tag::total.size::appsFor()::witnessed.size::unreachable.size::timeout.size::Nil
        def add(r:JoinedResultRow):CategorySummaryRow = {
            val cfg = r.getConfig
            assert(total.exists{other => other.getConfig == cfg})
            val newTimeout:Set[JoinedResultRow] = timeout.filter{other => 
                if(other.getConfig == cfg){
                    assert(other.summary == "unknown", s"duplicate row")
                }
                other.getConfig != cfg
            }
            assert(newTimeout.size == timeout.size - 1, s"timeouts did not contain row: ${write[RunConfig](cfg)}")
            if(r.summary.contains("Witnessed")){
                this.copy(witnessed = witnessed + r, timeout = newTimeout)
            }else if(r.summary.contains("Unreachable")){
                this.copy(unreachable = unreachable + r, timeout = newTimeout)
            }else if(r.summary.contains("Timeout")){
                this.copy(timeout = newTimeout + r)
            }else if(r.summary.contains("Error")){
                this.copy(timeout = newTimeout + r)
            }else{
                println(r)
                ???
            }
        }
    }


    def tagCount() = {
        implicit val getTagCount = GetResult(r => TagCount(r.<<, r.<<))
        val q = sql"select cast(config::json->'tag'->'heuristicType' as varchar) as tag,count(*) from jobs group by cast(config::json->'tag'->'heuristicType' as varchar)"
        Await.result(db.db.run(q.as[TagCount]), Duration(30,"seconds"))
    }

    def getRunConfigs() = {
        case class ConfigMaker(v:String){
            def apply():RunConfig = read[RunConfig](v)
        }
        implicit val getRunConfig = GetResult(r => ConfigMaker(r.<<))
        // implicit val getRunConfig = GetResult(r => read[RunConfig](r.<<))
        val q = sql"select cast(config as varchar) from jobs;"
        val res = Await.result(db.db.run(q.as[ConfigMaker]), Duration(60,"seconds")).map{_()}.toSet
        res
    }
    val categoryTable = {
        // put everything from tag count into a category summary row
        // initially set everything as timeout then move things to other categories as we iterate over defined results
        val allInTo:Map[String,CategorySummaryRow] = getRunConfigs().groupBy(_.tag.heuristicType).map{
            case (tag:String, configs) => 
                val configSet = configs.toSet.map{cfg => 
                    JoinedResultRow.fromConfig(cfg)
                }
                (tag, CategorySummaryRow(tag, total = configSet, witnessed = Set.empty, unreachable = Set.empty, timeout = configSet))
        }
        topResultsByQuery.foldLeft(allInTo){
            case (acc, v) => 
                val heuristic = ujson.read(v.tag)("heuristicType").str
                val current = acc(heuristic)
                val next = current.add(v)
                acc + (heuristic -> next)
        }
    }.toSeq.map{case (_,v) => v}
    val locCountMap = {
        val f = File("locCount.csv")
        f.contentAsString.split('\n').map{v => 
            val spl = v.split(',').toList
            (spl.head,spl.last.toInt)
        }.toList
    }
    def locCount(path:String):Int = {
        val trimmed = if(path.startsWith("/")){
            path.split("/").last
        }else{
            path.replace("${baseDir}","").drop(1).dropRight(1)
        }
        val find = locCountMap.filter{case (k,v) => k.contains(trimmed) }
        assert(find.size == 1, s"found: ${find.size} results for ${trimmed}")
        find.head._2
    }
    
}

defined class JoinedResultRow
defined object JoinedResultRow
defined class SummaryRow
defined class ApkSummaryRow
getRowResult: AnyRef with GetResult[JoinedResultRow] = <function1>
defined object ReadResults

Table 2
-------
The following corresponds to Table 2 alarm timeout safe and locations from the historia paper.

In [6]:
println(s"number of apps: ${ReadResults.appList.size}")
println(s"total lines of code: ${ReadResults.appList.map{a => ReadResults.locCount(a)}.sum}")
Table(ReadResults.categoryTable.map{r => r.copy(tag = r.tag match{ // map bad names chosen while running experiments to better names in paper
    case "Disallow.I_CIEnter_AsyncTaskexecute" => "execute[5]"
    case "SensitiveDerefFieldCausedSynch" => "subs null"
    case "SensitiveDerefFieldCausedFinish" => "finish null"
    case "SensitiveDerefCallinCaused" => "getAct[3]"
    case "Disallow.I_CIEnter_Dialogdismiss" => "dismiss[7]"
})})

number of apps: 9
total lines of code: 95790


tag,total,apps,witnessed,unreachable,timeout
execute[5],1,1,0,1,0
subs null,2,2,0,2,0
getAct[3],6,5,4,2,0
dismiss[7],1,1,0,1,0


In [7]:
def sampleByTag(tag:String) = {
    import scala.concurrent.duration.Duration
    case class StringResult(config:String, summary:String) extends TableAble{
          def headers: List[String] = "summary"::Nil
          def values: List[Any] = summary::Nil
    }
    implicit val getStringResult = GetResult(r => StringResult(r.<<, r.<<))
    // val q = sql"select config from jobs where cast(config::json->'tag'->'heuristicType' as varchar) like '%${tag}%' limit 1;"
    val q = sql"""select jobs.config, cast(results.result::json->'summary' as varchar) from jobs inner join results on jobs.id=results.jobid where cast(results.result::json->'summary' as varchar) not like '%Unreachable%' and cast(results.result::json->'summary' as varchar) not like '%Exception%' and cast(jobs.config::json->'tag'->'heuristicType' as varchar) like ${tag} order by random() limit 20"""
    Await.result(db.db.run(q.as[StringResult]), Duration(30,"seconds"))
}

//val sample = sampleByTag("%SensitiveDerefFieldCausedFinish%")

defined function sampleByTag

In [8]:
Table(ReadResults.topByApp)

apk,witnessed,unreachable,timeout,err
"""${baseDir}/fdroid/be.digitalia.fosdem/2.1.1/apk/be.digitalia.fosdem_1900211.apk""",0,1,0,0
"""${baseDir}/fdroid/com.darshancomputing.BatteryIndicatorPro/12.0.0/apk/com.darshancomputing.BatteryIndicatorPro_26016.apk""",1,0,0,0
"""${baseDir}/fdroid/com.nutomic.syncthingandroid/1.23.0/apk/com.nutomic.syncthingandroid_4334.apk""",0,1,0,0
"""${baseDir}/fdroid/de.blinkt.openvpn/0.7.43/apk/de.blinkt.openvpn_198.apk""",2,0,0,0
"""${baseDir}/fdroid/de.jkliemann.parkendd/1.2.8/apk/de.jkliemann.parkendd_33.apk""",0,1,0,0
"""${baseDir}/fdroid/de.markusfisch.android.wavelines/1.13.2/apk/de.markusfisch.android.wavelines_22.apk""",0,1,0,0
"""${baseDir}/fdroid/de.syss.MifareClassicTool/4.0.4/apk/de.syss.MifareClassicTool_62.apk""",0,1,0,0
"""${baseDir}/fdroid/dev.ukanth.ufirewall/3.5.3/apk/dev.ukanth.ufirewall_20220615.apk""",0,1,0,0
"""${baseDir}/fdroid/net.wigle.wigleandroid/2.63/apk/net.wigle.wigleandroid_263.apk""",1,0,0,0


In [9]:
// Code that randomly chooses samples to specify
object SamplesToSpecify{
    // choose 1 witness sample per app
    def toInt(s: String): Option[Int] = {
      try {
        Some(s.toInt)
      } catch {
        case e: Exception => None
      }
    }
    def createHeuristicDir(cfg:RunConfig) = {
        val appDirName:String = cfg.outFolder.get.split("/").last
        val appDir = expDir / appDirName
        appDir.createIfNotExists(asDirectory=true)
        val heuristicDir = appDir / cfg.tag.heuristicType
        heuristicDir.createIfNotExists(asDirectory=true)
        heuristicDir
    }
    def writeSamples(count:Int) = {
        // val samples = ReadResults.appList.map(app => ReadResults.sampleWitForTop(app)) //TODO: random choice needs to be moved so it gets more than one per app per run
        val samples = ReadResults.topByApp.flatMap{appRes => 
            Random.shuffle(appRes.row.witnessed).take(count)
        }
        samples.zipWithIndex.foreach{
            case (sample,ind) => 
                val cfg = sample.getConfig
                val heuristicDir = createHeuristicDir(cfg)
                val existingDir = heuristicDir.glob("*").toList.filter(a => !a.name.startsWith("."))
                val existingConfigs = heuristicDir.glob("**/config.json").toList.map(v => read[RunConfig](v.contentAsString))
                val currentCfg = read[RunConfig](sample.config).copy(initialQuery=List(read[InitialQuery](sample.qry)))
                if(existingDir.size < count){
                    if(existingConfigs.forall(_ != currentCfg)){
                        val nextNum = if(existingDir.isEmpty) 0 else{ existingDir.flatMap(v => toInt(v.name)).max + 1 }
                        val sampleDir = heuristicDir / nextNum.toString
                        sampleDir.createIfNotExists(asDirectory=true)
                        (sampleDir / "config.json").overwrite(write[RunConfig](currentCfg))
                    }
                }
                
                // val f = expDir / s"sample_${ind}"
                // assert(!f.exists())
                // f.createIfNotExists(asDirectory=true)
                // (f / "apk").overwrite(sample.apk)
                // (f / "summary").overwrite(sample.summary)
                // (f / "configTop.json").overwrite(write[RunConfig](read[RunConfig](sample.config).copy(initialQuery=List(read[InitialQuery](sample.qry)))))
        }
    }
}
//SamplesToSpecify.writeSamples(4)
println(s"Samples to specify: ${expDir.glob("**/config.json").toList.size}")

Samples to specify: 8


defined object SamplesToSpecify

In [10]:
import edu.colorado.plv.bounder.lifestate.FragmentGetActivityNullSpec
import edu.colorado.plv.bounder.lifestate.LifeState.{AbsMsg, LSSpec, LSPred, OAbsMsg, SetSignatureMatcher, Signature, SignatureMatcher, SubClassMatcher,Or,And,NS,Not}
import edu.colorado.plv.bounder.symbolicexecutor.state.{BoolVal, Equals, NamedPureVar, NotEquals, NullVal, PureExpr, TopVal}


import edu.colorado.plv.bounder.ir.{CBEnter, CBExit, CIExit}

object ToSpecify{
    println(expDir)
    def allCfg = (expDir / "SpecGen").glob("**/config.json").map{f => (f,read[RunConfig](f.contentAsString))}.toList
    val preRunCfg = (expDir / "data").glob("**/config.json").map{f => (f,read[RunConfig](f.contentAsString))}.toList
    
}
val f = NamedPureVar("f")
val b = NamedPureVar("b")
val l = NamedPureVar("l")
val a = NamedPureVar("a")
val p = NamedPureVar("p")
println(ToSpecify.allCfg.size)

case class AppCitation(name:String, date:DateTime, url:String, pkg:String){
    
}
object AppCitation{
    val citations = List(
        AppCitation("Seafile",(new DateTime).withYear(2022).withMonthOfYear(8).withDayOfMonth(23), 
                    "https://f-droid.org/en/packages/com.seafile.seadroid2/", "com.seafile.seadroid2"),
        AppCitation("BatteryBot", (new DateTime).withYear(2021).withMonthOfYear(12).withDayOfMonth(07),
            "https://f-droid.org/en/packages/com.darshancomputing.BatteryIndicatorPro/","com.darshancomputing.BatteryIndicatorPro"),
        AppCitation("Vanilla Music", (new DateTime).withYear(2023).withMonthOfYear(2).withDayOfMonth(18), 
                    "https://f-droid.org/en/packages/ch.blinkenlights.android.vanilla/", "ch.blinkenlights.android.vanilla"),
        AppCitation("Syncthing", (new DateTime).withYear(2023).withMonthOfYear(01).withDayOfMonth(08),
            "https://f-droid.org/en/packages/com.nutomic.syncthingandroid/","com.nutomic.syncthingandroid"),
        AppCitation("OpenVPN", (new DateTime).withYear(2022).withMonthOfYear(12).withDayOfMonth(17),
                    "https://f-droid.org/en/packages/de.blinkt.openvpn/" ,"de.blinkt.openvpn"),
        AppCitation("Navit", (new DateTime).withYear(2021).withMonthOfYear(03).withDayOfMonth(11),
            "https://f-droid.org/en/packages/org.navitproject.navit/","org.navitproject.navit"),
        AppCitation("Connectbot", (new DateTime).withYear(2022).withMonthOfYear(8).withDayOfMonth(23),
                    "https://github.com/connectbot/connectbot", "org.connectbot"),

        AppCitation("Antennapod", (new DateTime).withYear(2023).withMonthOfYear(1).withDayOfMonth(1),
                    "https://github.com/AntennaPod/AntennaPod", "de.danoeh.antennapod")
    )
    def fromFolder(folder:String):AppCitation = {
        println(folder)
        citations.find(c => folder.contains(c.pkg)).getOrElse(throw new Exception(s"citation not found for folder: ${folder}"))
    }
}
object Location{
    def apply(i:InitialQuery):String = {
        val fullClass = i match{
            case i:ReceiverNonNull => i.sig.base
            case i:DisallowedCallin => i.className
            case _ => ???
        }
        val spl = fullClass.split('.')
        if(spl.isEmpty){
            throw new RuntimeException(s"could not parse initial query: $i")
        }
        
        val className = spl.last
        className.split('$').head
    }
}

trait Specifyable[T<:InitialQuery] extends TableAble{
    def initial:T
    val firstRunFind = {
        val witnessed = ReadResults.topByApp.flatMap{v => v.row.witnessed}
        val unreachable = ReadResults.topByApp.flatMap{v => v.row.unreachable}
        val err = ReadResults.topByApp.flatMap{v => v.row.err}
        def contains(row: Iterable[JoinedResultRow]):Boolean = {
            row.exists(_.getConfig.initialQuery.contains(initial))
        }
        if(contains(witnessed)){
            assert(!contains(err) && !contains(unreachable))
            "witnessed"
        }else if(contains(unreachable)){
            assert(!contains(err))
            "unreachable"
        }else if(contains(err)){
            "err"
        }else{
            "timeout"
        }
    }
    def configWithSpec:RunConfig
    def reachableDual:InitialQuery
    lazy val mConfig = ToSpecify.allCfg.filter{case (_,cfg) => cfg.initialQuery.exists(v => v == initial)}
    lazy val configFile = mConfig.head._1
    def getCitation = {
        AppCitation.fromFolder(configFile.toString)
    }
    lazy val getMsg = {
        val rootDir = configFile.parent
        val msgFolder = rootDir / "spec" // change to messages_all.json for full non-filtered
        val toRead = msgFolder.glob("**/messages_component.json").toList
        if(toRead.size == 1){
            val msgFile = toRead.toList.head
            val out = read[Messages](msgFile.contentAsString)
            out
        }else {
             Messages(-1, -1, -1, -1,-1, -1,-1, -1)   
        }
    }
    lazy val getAllMsg = {
        val rootDir = configFile.parent
        val msgFolder = rootDir / "spec" // change to messages_all.json for full non-filtered
        val toRead = msgFolder.glob("**/messages_all.json").toList
        if(toRead.size == 1){
            val msgFile = toRead.toList.head
            val out = read[Messages](msgFile.contentAsString)
            out
        }else {
             Messages(-1, -1, -1, -1,-1, -1,-1, -1)   
        }  
    }
    lazy val getRes = {
        val rootDir = configFile.parent
        val msgFolder = rootDir / "spec"
        val resFile = msgFolder.glob("**/result_0.txt").toList
        if(resFile.size == 1){
            ujson.read(resFile.head.contentAsString)
        }else ujson.read("{}")
    }
    def getConfigWithSpecPath:File = configFile.parent / "config_spec.json"
    def runHistoriaWithSpec():Driver.LocResult = {
        val javaMemLimit=8 // Gb Note that this only limits JVM not JNI which can go significantly higher
        val historiaJar = "/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
        val apkRootDir = "/home/historia_generalizability"
        val outDir = getConfigWithSpecPath.parent.toString
        println(s"out directory: ${outDir}")
        val config = read[RunConfig](getConfigWithSpecPath.contentAsString)
        println(s"apk path: ${config.apkPath}")
        val outSubdir = config.outFolder.get.replace("${baseDirOut}",outDir)
        println(s"out subdirectory: ${config.outFolder}")
        val cmd = s"java -Xmx${javaMemLimit}G -jar ${historiaJar} -m verify -c ${getConfigWithSpecPath} -b ${apkRootDir} -u ${outDir} -o MEM --debug"
        BounderUtil.runCmdStdout(cmd)

        // val infoCmd = cmd.replace("verify","info") // uncomment to recompute message info for table 3
        // val outDb = outSubdir / "out.db"
        // if(outDb.exists){
        //     outDb.delete()
        // }
        
        // BounderUtil.runCmdStdout(infoCmd)

        // println((outSubdir / "messages_component.json").contentAsString)

        read[Driver.LocResult]((outSubdir / "result_0.txt").contentAsString)
    }
    def writeSpec():Unit = {
        // verify with added specification
        val outf = getConfigWithSpecPath
        outf.overwrite(write(configWithSpec))
        
        //check that line is reachable under normal circumstances
        val outfReach = configFile.parent / "config_spec_locreach.json"
        outfReach.overwrite(write(configWithSpec.copy(initialQuery = List(reachableDual))))
    }
    def result:String
    def specificationTime:Duration
    def readableSpecificationTime:String = {
        scala.math.BigDecimal(specificationTime.minutes).setScale(0,scala.math.BigDecimal.RoundingMode.HALF_UP).toInt.toString
    }
  lazy val appT = getCitation.name
  lazy val appCbMsgT = getMsg.cbSize
   
  lazy val texTag = configWithSpec.tag.heuristicType match{
      case "SensitiveDerefCallinCaused" => "\\apGa"
      case "Disallow.I_CIEnter_AsyncTaskexecute" => "\\apEx"
      case "Disallow.I_CIEnter_Dialogdismiss" => "\\ymDi"
      case "SensitiveDerefFieldCausedFinish" => "\\cbFi"
      case "SensitiveDerefFieldCausedSynch" => "\\synch"
      case v => 
         throw new RuntimeException(s"did not find tex version of tag: ${v}")
  }
  lazy val columns = List(
      ("app", getCitation.name),
      ("location", Location(initial)),
      ("pattern", texTag),
      ("cb/cbret", getAllMsg.cbSize),
      ("ci", getAllMsg.syntCi),
      ("filt cb/cbret", getMsg.cbSize),
      ("filt ci", getMsg.syntCi),
      ("specs", configWithSpec.specSet.getSpecSet.size + (initial match{
          case _:ReceiverNonNull => 0
          case _:DisallowedCallin => 1
          case _ => throw new RuntimeException(s"unsupported initial: ${initial}")
      })),
      ("cb", getMsg.matchedCb),
      ("cb\\%", scala.math.BigDecimal(getMsg.matchedCb.toDouble / getMsg.cbSize.toDouble * 100).setScale(0,scala.math.BigDecimal.RoundingMode.HALF_UP).toString),
      ("cbret", getMsg.matchedCbRet),
      ("cbret\\%", scala.math.BigDecimal(getMsg.matchedCbRet.toDouble / getMsg.cbSize.toDouble * 100).setScale(0,scala.math.BigDecimal.RoundingMode.HALF_UP).toString),
      ("ci", getMsg.matchedSyntCi),
      ("ci\\%", scala.math.BigDecimal(getMsg.matchedSyntCi.toDouble / getMsg.syntCi.toDouble * 100).setScale(0,scala.math.BigDecimal.RoundingMode.HALF_UP).toString),
      ("time", if(result == "timeout") "" else getRes("time").num.toInt),
      //("depth", if(result == "timeout") "" else getRes("depthChar")("cbDepth").num.toInt),
      ("res", result),
      ("spec time", readableSpecificationTime)
      ).unzip
  def headers:List[String] = columns._1
  def values:List[Any]     = columns._2
}

/home/notebooks
8


import edu.colorado.plv.bounder.lifestate.FragmentGetActivityNullSpec

import edu.colorado.plv.bounder.lifestate.LifeState.{AbsMsg, LSSpec, LSPred, OAbsMsg, SetSignatureMatcher, Signature, SignatureMatcher, SubClassMatcher,Or,And,NS,Not}

import edu.colorado.plv.bounder.symbolicexecutor.state.{BoolVal, Equals, NamedPureVar, NotEquals, NullVal, PureExpr, TopVal}



import edu.colorado.plv.bounder.ir.{CBEnter, CBExit, CIExit}


defined object ToSpecify
f: NamedPureVar = NamedPureVar("f")
b: NamedPureVar = NamedPureVar("b")
l: NamedPureVar = NamedPureVar("l")
a: NamedPureVar = NamedPureVar("a")
p: NamedPureVar = NamedPureVar("p")
defined class AppCitation
defined object AppCitation
defined object Location
defined trait Specifyable

Table 3 Data - Write Specifications For Individual Samples
===========================================

The following cells contain one location each corresponding to rows of Table 3.
Each cell defines an object such as `Specify_0` containing the info to run Historia on one of the manually inspected locations.
The name of the app and corresponding row of the table will be printed when each cell is run.
These were selected from the alarms and timeouts from the previous experiment.
We attempted to write CBCFTL specifications for each location, these are defined in the `configWithSpec` field.
Each cell also defines a `result` field, we suggest not running the verifier on these as it takes a long time and is not particularly interesting.

To run the verifier, uncomment the `runHistoriaWithSpec` method call and print statement the end of the corresponding cell.  The verification result will be printed below upon completion.




In [11]:

case object Specify_0 extends Specifyable[ReceiverNonNull]{
    val result = "timeout" //This one may possibly be an alarm but historia times out.
    val initial = read[InitialQuery]("""
        {
          "methodName": "void onClick(android.content.DialogInterface,int)",
          "line": 297,
          "t": "ReceiverNonNull",
          "matcher": "void onBackPressed()",
          "className": "ch.blinkenlights.android.vanilla.PreferencesActivity$HeadsetLaunchFragment$2"
        }
      """).asInstanceOf[ReceiverNonNull]
    val reachableDual = Reachable(initial.sig, initial.line)

    //private val mConfig = ToSpecify.allCfg.filter{case (_,cfg) => cfg.initialQuery.exists(v => v == initial)}
    assert(mConfig.size == 1)
    val config = mConfig.head._2.copy(componentFilter = Some(Seq(
        // "ch.blinkenlights.android.vanilla.PreferencesActivity.*",
        //                                                          "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$HelpFragment",
        //                                                          "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$EqualizerFragment", 
        //                                                          "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$PlaylistFragment",
        //                                                          "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$PlaybackFragment",
        //                                                          "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$ReplayGainFragment", 
        //                                                          "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$CoverArtFragment", 
        //                                                          "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$NotificationsFragment", 
        //                                                          "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$MiscFragment",
        //                                                          "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$LibraryFragment",
        //                                                          "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$AboutFragment",
        //                                                          "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$AudioFragment",
        //                                                          "!ch.blinkenlights.android.vanilla.PreferencesActivity\\$ShakeFragment",
                                                                )))
    // val configFile = mConfig.head._1
    
    val Fragment_onCreate: SignatureMatcher = SubClassMatcher(SpecSignatures.Fragment,
          "void onCreate\\(android.os.Bundle\\)", "Fragment_onCreate")
                                                                                                             
    val Fragment_onCreate_entry: OAbsMsg = AbsMsg(CBEnter, Fragment_onCreate, TopVal::f::Nil)
    val fragmentActivityNotAttached_improved: LSPred =
        Or(And(
            NS(SpecSignatures.Fragment_onDestroy_exit, SpecSignatures.Fragment_onActivityCreated_entry),
            NS(SpecSignatures.Fragment_onDestroy_exit, Fragment_onCreate_entry)
        ),
          And(Not(SpecSignatures.Fragment_onActivityCreated_entry), Not(Fragment_onCreate_entry)))
    val fragment_resumed:LSPred = NS(Fragment_onCreate_entry, SpecSignatures.Fragment_onDestroy_exit)
    val fragment_create_only_once:LSSpec = LSSpec(f::Nil, Nil, Not(Fragment_onCreate_entry), Fragment_onCreate_entry)
    val DialogBuilder = Set("android.app.AlertDialog$Builder")
    val DialogBuilder_init = AbsMsg(CIExit,SubClassMatcher(DialogBuilder, "void <init>\\(android.content.Context\\)", "DialogBuilder_init"), TopVal::b::a::Nil)
    val DialogBuilder_set__Button= (negPos:String) => AbsMsg(CIExit, 
                                                         SubClassMatcher(DialogBuilder,
                                                         "android.app.AlertDialog\\$Builder set" + negPos + "Button\\(int,android.content.DialogInterface\\$OnClickListener\\)",
                                                         s"DialogBuilder_set${negPos}Button"), TopVal::b::TopVal::l::Nil)
    val DialogBuilder_setNegativeButton = DialogBuilder_set__Button("Negative")
    val DialogBuilder_setPositiveButton = DialogBuilder_set__Button("Positive")
    val DialogOnClick_onClick = AbsMsg(CBEnter,
                                       SubClassMatcher(Set("android.content.DialogInterface$OnClickListener"),
                                                       "void onClick\\(android.content.DialogInterface,int\\)",
                                                       "DialogOnClick_onClick"), TopVal::l::Nil)
    
    val Dialog_onClick_reg = LSSpec(l::Nil,a::b::f::Nil,
      And(And(
          And(SpecSignatures.Fragment_get_activity_exit,DialogBuilder_init), 
          fragment_resumed),
      Or(DialogBuilder_setNegativeButton,DialogBuilder_setPositiveButton)), DialogOnClick_onClick)
    val FragmentGetActivityNullSpec_improved = FragmentGetActivityNullSpec.getActivityNull.copy(pred = fragmentActivityNotAttached_improved)
    val configWithSpec = config.copy(specSet = config.specSet.asInstanceOf[PickleSpec]
                                     .copy(specs = 
                                           Set(
                                               fragment_create_only_once,
                                               FragmentGetActivityNullSpec_improved,
                                               Dialog_onClick_reg
                                              )))
//     def writeSpec():Unit = {
//         // verify with added specification
//         val outf = configFile.parent / "config_spec.json"
//         outf.overwrite(write(configWithSpec))
        
//         //check that line is reachable under normal circumstances
//         val outfReach = configFile.parent / "config_spec_locreach.json"
//         outfReach.overwrite(write(configWithSpec.copy(initialQuery = List(reachableDual))))
//     }
    val specificationTime = 
        (20.minutes +  3.seconds + 
        42.minutes + 28.seconds +
         9.minutes + 15.seconds +
         8.minutes + 32.seconds +
         3.minutes +  9.seconds).toDuration
    //(20 min 3 seconds so far) + (42:28) + (9:15) + (8:32) + (3:09)
}
// Specify_0.writeSpec()
Specify_0.configFile
println(s"App Name: ${Specify_0.appT}")

//Uncomment the following two lines to run Historia on this sample from Table 3
// val result = Specify_0.runHistoriaWithSpec()
// println(s"Result Summary: ${result.resultSummary}")


/home/notebooks/SpecGen/ch.blinkenlights.android.vanilla/SensitiveDerefCallinCaused/0/config.json
App Name: Vanilla Music


defined object Specify_0
res10_1: File = /home/notebooks/SpecGen/ch.blinkenlights.android.vanilla/SensitiveDerefCallinCaused/0/config.json

In [12]:
case object Specify_1 extends Specifyable[ReceiverNonNull]{ // (3:25)
    val path = "bounder/notebooks/ossExp/SpecGen/com.darshancomputing.BatteryIndicatorPro/SensitiveDerefCallinCaused/0"
    val initial = read[InitialQuery]("""
    {
      "methodName": "void loadSettingsFiles()",
      "line": 185,
      "t": "ReceiverNonNull",
      "matcher": "android.content.SharedPreferences getSharedPreferences(java.lang.String,int)",
      "className": "com.darshancomputing.BatteryIndicatorPro.PersistentFragment"
    }
  """).asInstanceOf[ReceiverNonNull]
    val reachableDual = Reachable(initial.sig, initial.line)
    // private val mConfig = ToSpecify.allCfg.filter{case (_,cfg) => cfg.initialQuery.exists(v => v == initial)}
    assert(mConfig.size == 1)
    val config = mConfig.head._2
    val configWithSpec = config.copy(specSet = config.specSet.asInstanceOf[PickleSpec]
                                     .copy(specs = Set(FragmentGetActivityNullSpec.getActivityNull.copy(pred = Specify_0.fragmentActivityNotAttached_improved))))
    val result = "proven with spec"//checked 3/25/23 , 3/28/23, 3/31/23
    val specificationTime = 
        (3.minutes + 25.seconds).toDuration
}
Specify_1.writeSpec()
Specify_1.configFile

println(s"App Name: ${Specify_1.appT}")

//Uncomment the following two lines to run Historia on this sample from Table 3
// val result = Specify_1.runHistoriaWithSpec()
// println(s"Result Summary: ${result.resultSummary}")



/home/notebooks/SpecGen/com.darshancomputing.BatteryIndicatorPro/SensitiveDerefCallinCaused/0/config.json
App Name: BatteryBot


defined object Specify_1
res11_2: File = /home/notebooks/SpecGen/com.darshancomputing.BatteryIndicatorPro/SensitiveDerefCallinCaused/0/config.json

In [13]:
case object Specify_2 extends Specifyable[ReceiverNonNull]{ // (9:46) //TODO: see what alarm is with this and iterate further
    override def initial:ReceiverNonNull = read[InitialQuery]("""
    {
      "methodName": "void lambda$onPreferenceClick$9$com-nutomic-syncthingandroid-activities-SettingsActivity$SettingsFragment(android.content.Intent,android.content.DialogInterface,int)",
      "line": 638,
      "t": "ReceiverNonNull",
      "matcher": "android.content.ComponentName startService(android.content.Intent)",
      "className": "com.nutomic.syncthingandroid.activities.SettingsActivity$SettingsFragment"
    }
    """).asInstanceOf[ReceiverNonNull]
    val reachableDual = Reachable(initial.sig, initial.line)
    assert(mConfig.size == 1)
    val config = mConfig.head._2.copy(componentFilter=Some(Seq("com.nutomic.syncthingandroid.activities.SettingsActivity\\$SettingsFragment")))
    val configWithSpec = config.copy(specSet = Specify_0.configWithSpec.specSet)
    val result = "timeout"
    val specificationTime = 
        (9.minutes + 46.seconds).toDuration

}
Specify_2.writeSpec()
Specify_2.configFile
println(s"App Name: ${Specify_2.appT}")

//Uncomment the following two lines to run Historia on this sample from Table 3
// val result = Specify_2.runHistoriaWithSpec()
// println(s"Result Summary: ${result.resultSummary}")

/home/notebooks/SpecGen/com.nutomic.syncthingandroid/SensitiveDerefCallinCaused/0/config.json
App Name: Syncthing


defined object Specify_2
res12_2: File = /home/notebooks/SpecGen/com.nutomic.syncthingandroid/SensitiveDerefCallinCaused/0/config.json

In [22]:
case object Specify_3 extends Specifyable[ReceiverNonNull]{ // 4:55 + :55
    //ossExp/SpecGen/com.seafile.seadroid2/SensitiveDerefCallinCaused/0
    val initial = read[InitialQuery]("""
    {
      "methodName": "android.app.Dialog onCreateDialog(android.os.Bundle)",
      "line": 68,
      "t": "ReceiverNonNull",
      "matcher": "android.view.LayoutInflater getLayoutInflater()",
      "className": "com.seafile.seadroid2.ui.dialog.SslConfirmDialog"
    }
    """).asInstanceOf[ReceiverNonNull]
    val reachableDual = Reachable(initial.sig, initial.line)
    assert(mConfig.size == 1, s"wrong config file count: ${mConfig.size}")
    val config = mConfig.head._2
    val DialogFragment = Set("android.support.v4.app.DialogFragment")
    val Fragment_onCreateDialog = AbsMsg(CBEnter, 
                                         SubClassMatcher(DialogFragment,"android.app.Dialog onCreateDialog\\(android.os.Bundle\\)", "Fragment_onCreateDialog"), 
                                         TopVal::f::Nil)
    val Fragment_resumed = And(
        NS(Specify_0.Fragment_onCreate_entry, SpecSignatures.Fragment_onDestroy_exit),
        NS(SpecSignatures.Fragment_onActivityCreated_entry, SpecSignatures.Fragment_onDestroy_exit))
    val DialogFragment_onCreateDialog_whenCreated = LSSpec(f::Nil, Nil, Fragment_resumed, 
                                                           Fragment_onCreateDialog)
    val configWithSpec = config.copy(specSet = PickleSpec(specs = 
                                                          Set(DialogFragment_onCreateDialog_whenCreated,  Specify_0.FragmentGetActivityNullSpec_improved), 
                                                          disallow=Set(), 
                                                          matcherSpace=Specify_0.configWithSpec.specSet.getSpecSpace().allI + Fragment_onCreateDialog))
    val manualNoSpecRun = "alarm"
    val result = "proven with spec" //checked 3/31/23
    val specificationTime = (4.minutes + 55.seconds + 
                             55.seconds).toDuration
}
Specify_3.writeSpec()
Specify_3.configFile
println(s"App Name: ${Specify_3.appT}")

//Uncomment the following two lines to run Historia on this sample from Table 3
// val result = Specify_3.runHistoriaWithSpec()
// println(s"Result Summary: ${result.resultSummary}")


/home/notebooks/SpecGen/com.seafile.seadroid2/SensitiveDerefCallinCaused/0/config.json
App Name: Seafile
out directory: /home/notebooks/SpecGen/com.seafile.seadroid2/SensitiveDerefCallinCaused/0
apk path: ${baseDir}/fdroid/com.seafile.seadroid2/2.2.48/apk/com.seafile.seadroid2_123.apk
out subdirectory: Some(${baseDirOut}/com.seafile.seadroid2)
Result Summary: Proven


defined object Specify_3
res21_2: File = /home/notebooks/SpecGen/com.seafile.seadroid2/SensitiveDerefCallinCaused/0/config.json
result: Driver.LocResult = LocResult(
  ReceiverNonNull(
    Signature(
      "com.seafile.seadroid2.ui.dialog.SslConfirmDialog",
      "android.app.Dialog onCreateDialog(android.os.Bundle)"
    ),
    68,
    Some("android.view.LayoutInflater getLayoutInflater()")
  ),
  -1,
  AppLoc(
    SerializedIRMethodLoc(
      "com.seafile.seadroid2.ui.dialog.SslConfirmDialog",
      "android.app.Dialog onCreateDialog(android.os.Bundle)",
      List(
        Some(
          LocalWrapper(
            "@this",
            "com.seafile.seadroid2.ui.dialog.SslConfirmDialog"
          )
        ),
        Some(LocalWrapper("@parameter0", "android.os.Bundle"))
      )
    ),
    SerializedIRLineLoc(
      68,
      "line: 68 $r6 = virtualinvoke $r4.<android.app.Activity: android.view.LayoutInflater getLayoutInflater()>()",
      1064985695
    ),
    true
  ),
  Proven,
  Mu

In [15]:
case object Specify_4 extends Specifyable[ReceiverNonNull]{ // 5:16
    val initial = read[InitialQuery]("""
    {
      "methodName": "void onCreateOptionsMenu(android.view.Menu,android.view.MenuInflater)",
      "line": 345,
      "t": "ReceiverNonNull",
      "matcher": "java.lang.String getString(int)",
      "className": "de.blinkt.openvpn.fragments.VPNProfileList"
    }
    """).asInstanceOf[ReceiverNonNull]
    val reachableDual = Reachable(initial.sig, initial.line)
    assert(mConfig.size == 1)
    val config = mConfig.head._2
    
    val Fragment_onCreateOptionsMenu = AbsMsg(CBEnter, SubClassMatcher(SpecSignatures.Fragment, 
                                                                       "void onCreateOptionsMenu\\(android.view.Menu,android.view.MenuInflater\\)",
                                                                       "Fragment_onCreateOptionsMenu"), TopVal::f::Nil)
    
    def whileFragmentResumed(msg:OAbsMsg):LSSpec = {
        LSSpec(f::Nil, Nil, Specify_3.Fragment_resumed, msg)
    }
    val Fragment_onCreateOptionsMenu_whileResumed = whileFragmentResumed(Fragment_onCreateOptionsMenu)
                                              
    val configWithSpec = config.copy(specSet = PickleSpec(specs = 
                                                          Set(Fragment_onCreateOptionsMenu_whileResumed,  
                                                              Specify_0.FragmentGetActivityNullSpec_improved), 
                                                          disallow=Set(), 
                                                          matcherSpace=Specify_0.configWithSpec.specSet.getSpecSpace().allI + Specify_3.Fragment_onCreateDialog))
    val result = "proven with spec" // checked 3/31/23
    val specificationTime = 
        (5.minutes + 16.seconds).toDuration
}
Specify_4.writeSpec()
Specify_4.configFile

//Uncomment the following two lines to run Historia on this sample from Table 3
// val result = Specify_4.runHistoriaWithSpec()
// println(s"Result Summary: ${result.resultSummary}")

println(s"App Name: ${Specify_4.appT}")


/home/notebooks/SpecGen/de.blinkt.openvpn/SensitiveDerefCallinCaused/0/config.json
App Name: OpenVPN


defined object Specify_4
res14_2: File = /home/notebooks/SpecGen/de.blinkt.openvpn/SensitiveDerefCallinCaused/0/config.json

In [16]:
case object Specify_5 extends Specifyable[ReceiverNonNull]{ //9:44 + 7:07 + 9:56 + 8:27
    val initial = read[InitialQuery]("""
    {
      "methodName": "boolean lambda$setupScreen$1$SynchronizationPreferencesFragment(androidx.preference.Preference)",
      "line": 96,
      "t": "ReceiverNonNull",
      "matcher": "android.content.Context getApplicationContext()",
      "className": "de.danoeh.antennapod.fragment.preferences.synchronization.SynchronizationPreferencesFragment"
    }
    """).asInstanceOf[ReceiverNonNull]
    val reachableDual = Reachable(initial.sig, initial.line)
    assert(mConfig.size == 1)
    val config = mConfig.head._2.copy(
        componentFilter = Some(Seq("de.danoeh.antennapod.fragment.preferences.synchronization.SynchronizationPreferencesFragment.*")))
    val Preference_setOnPreferenceClickListener = AbsMsg(CIExit, 
                                                         SubClassMatcher(
                                                             Set("androidx.preference.Preference"), 
                                                             "void setOnPreferenceClickListener\\(androidx.preference.Preference\\$OnPreferenceClickListener\\)",
                                                             "Preference_setOnPreferenceClickListener"),
                                                         TopVal::p::l::Nil)
    val OnPreferenceClickListener_onClick = AbsMsg(CBEnter,
                                                   SubClassMatcher(
                                                       Set("androidx.preference.Preference$OnPreferenceClickListener"),
                                                       "boolean onPreferenceClick\\(androidx.preference.Preference\\)", 
                                                       "OnPreferenceClickListener_onClick"),
                                                   TopVal::l::Nil)
    val Fragment_findPreference = AbsMsg(CIExit,
                                         SubClassMatcher(
                                             SpecSignatures.Fragment,
                                             "androidx.preference.Preference findPreference\\(java.lang.CharSequence\\)",
                                             "Fragment_findPreference"),
                                         p::f::Nil)
    
    val Fragment_onCreatePreferences = AbsMsg(CBEnter,
                                              SubClassMatcher(
                                                  SpecSignatures.Fragment,
                                                  "void onCreatePreferences\\(android.os.Bundle,java.lang.String\\)",
                                                  "Fragment_onCreatePreferences"),
                                              TopVal::f::Nil)
                                             
    val Fragment_onCreatePreferences_whileResumed = Specify_4.whileFragmentResumed(Fragment_onCreatePreferences)
    
    val OnPreferenceClickListener_onClick_whileResumed = LSSpec(l::Nil,p::f::Nil,
                                                                And(Specify_3.Fragment_resumed, Fragment_findPreference),
                                                                OnPreferenceClickListener_onClick)
    
    val configWithSpec = config.copy(specSet = PickleSpec(specs = 
                                                          Set(Fragment_onCreatePreferences_whileResumed, 
                                                              Specify_0.FragmentGetActivityNullSpec_improved,
                                                              OnPreferenceClickListener_onClick_whileResumed
                                                             ), 
                                                          disallow=Set(), 
                                                          matcherSpace=Specify_0.configWithSpec.specSet.getSpecSpace().allI + 
                                                          Specify_3.Fragment_onCreateDialog + Specify_4.Fragment_onCreateOptionsMenu +
                                                          Preference_setOnPreferenceClickListener + OnPreferenceClickListener_onClick +
                                                          Fragment_findPreference + Fragment_onCreatePreferences
                                                         ))
    val result = "timeout" //TODO:  may be able to improve
    //9:44 + 7:07 + 9:56 + 8:27
    val specificationTime = 
        (9.minutes + 44.seconds +
         7.minutes +  7.seconds +
         9.minutes + 56.seconds +
         8.minutes + 27.seconds).toDuration
}
Specify_5.writeSpec()
Specify_5.configFile

//Uncomment the following two lines to run Historia on this sample from Table 3
// val result = Specify_5.runHistoriaWithSpec()
// println(s"Result Summary: ${result.resultSummary}")

println(s"App Name: ${Specify_5.appT}")

/home/notebooks/SpecGen/de.danoeh.antennapod/SensitiveDerefCallinCaused/0/config.json
App Name: Antennapod


defined object Specify_5
res15_2: File = /home/notebooks/SpecGen/de.danoeh.antennapod/SensitiveDerefCallinCaused/0/config.json

In [17]:
case object Specify_6 extends Specifyable[DisallowedCallin]{ // ./Disallow.I_CIEnter_Dialogdismiss_1.json 
    
    lazy val initial = read[InitialQuery]("""
    {
    "t": "DisallowedCallin",
    "className": "org.connectbot.GeneratePubkeyActivity$5",
    "methodName": "void run()",
    "s": "{\"univQuant\":[{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\",\"n\":\"d\"}],\"existQuant\":[{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\",\"n\":\"a\"}],\"pred\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.And\",\"l1\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.OAbsMsg\",\"mt\":{\"$type\":\"edu.colorado.plv.bounder.ir.CIExit\"},\"signatures\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.SubClassMatcher\",\"baseSubtypeOf\":[\"android.app.ProgressDialog\",\"android.app.Dialog\"],\"signatureMatcher\":\".*show.*\",\"ident\":\"Dialog_show\"},\"lsVars\":[{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\",\"n\":\"d\"},{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.TopVal\"},{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\",\"n\":\"a\"}]},\"l2\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.Or\",\"l1\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.Not\",\"p\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.OAbsMsg\",\"mt\":{\"$type\":\"edu.colorado.plv.bounder.ir.CBEnter\"},\"signatures\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.SubClassMatcher\",\"baseSubtypeOf\":[\"android.app.Activity\",\"androidx.fragment.app.FragmentActivity\"],\"signatureMatcher\":\"void onResume\\\\(\\\\)\",\"ident\":\"Activity_onResume\"},\"lsVars\":[{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.TopVal\"},{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\",\"n\":\"a\"}]}},\"l2\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.NS\",\"i1\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.OAbsMsg\",\"mt\":{\"$type\":\"edu.colorado.plv.bounder.ir.CBExit\"},\"signatures\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.SubClassMatcher\",\"baseSubtypeOf\":[\"android.app.Activity\",\"androidx.fragment.app.FragmentActivity\"],\"signatureMatcher\":\"void onPause\\\\(\\\\)\",\"ident\":\"Activity_onPause\"},\"lsVars\":[{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.TopVal\"},{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\",\"n\":\"a\"}]},\"i2\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.OAbsMsg\",\"mt\":{\"$type\":\"edu.colorado.plv.bounder.ir.CBEnter\"},\"signatures\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.SubClassMatcher\",\"baseSubtypeOf\":[\"android.app.Activity\",\"androidx.fragment.app.FragmentActivity\"],\"signatureMatcher\":\"void onResume\\\\(\\\\)\",\"ident\":\"Activity_onResume\"},\"lsVars\":[{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.TopVal\"},{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\",\"n\":\"a\"}]}}}},\"target\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.OAbsMsg\",\"mt\":{\"$type\":\"edu.colorado.plv.bounder.ir.CIEnter\"},\"signatures\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.SubClassMatcher\",\"baseSubtypeOf\":[\"android.app.ProgressDialog\",\"android.app.Dialog\"],\"signatureMatcher\":\"void dismiss\\\\(\\\\)\",\"ident\":\"Dialog_dismiss\"},\"lsVars\":[{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.TopVal\"},{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\",\"n\":\"d\"}]}}"
    }""").asInstanceOf[DisallowedCallin]
    assert(mConfig.size == 1)
    lazy val reachableDual = Reachable(Signature(initial.className, initial.methodName), 408)
    lazy val config = mConfig.head._2.copy(
        componentFilter = Some(Seq("org.connectbot.GeneratePubkeyActivity.*","org.connectbot.util.EntropyView.*", "org.connectbot.util.PubkeyDatabase.*")))
    lazy val configWithSpec = config.copy(specSet = PickleSpec(specs = Set(), disallow = config.specSet.getDisallowSpecSet()))
    val result = "true alarm"  // note that this was the one that is likely a real alarm
    val specificationTime = 
        (4.minutes + 30.seconds).toDuration
}
Specify_6.writeSpec()
Specify_6.configFile

//Uncomment the following two lines to run Historia on this sample from Table 3
val result = Specify_6.runHistoriaWithSpec()
println(s"Result Summary: ${result.resultSummary}"); println(s"Witness Explanation ${result.witnessExplanation}")
println(s"App Name: ${Specify_6.appT}")

out directory: /home/notebooks/SpecGen/org.connectbot/Disallow.I_CIEnter_Dialogdismiss/0
apk path: ${baseDir}/oss/org.connectbot/app/build/outputs/apk/oss/debug/app-oss-debug.apk
out subdirectory: Some(${baseDirOut}/org.connectbot)
Result Summary: Witnessed
Witness Explanation List(Future trace:
 TNew(p-0,{1})
CIExit show(( _T_,_T_ )
CIEnter void dismiss()( _T_,@3 )
)
/home/notebooks/SpecGen/org.connectbot/Disallow.I_CIEnter_Dialogdismiss/0/config.json
App Name: Connectbot


defined object Specify_6
res16_2: File = /home/notebooks/SpecGen/org.connectbot/Disallow.I_CIEnter_Dialogdismiss/0/config.json
result: Driver.LocResult = LocResult(
  DisallowedCallin(
    "org.connectbot.GeneratePubkeyActivity$5",
    "void run()",
    LSSpec(
      List(NamedPureVar("d")),
      List(NamedPureVar("a")),
      And(
        OAbsMsg(
          CIExit,
          SubClassMatcher(
            Set("android.app.ProgressDialog", "android.app.Dialog"),
            ".*show.*",
            "Dialog_show"
          ),
          List(NamedPureVar("d"), _T_, NamedPureVar("a"))
        ),
        Or(
          Not(
            OAbsMsg(
              CBEnter,
              SubClassMatcher(
                Set(
                  "android.app.Activity",
                  "androidx.fragment.app.FragmentActivity"
                ),
                "void onResume\\(\\)",
                "Activity_onResume"
              ),
              List(_T_, NamedPureVar("a"))
            )
          

In [18]:
case object Specify_7 extends Specifyable[ReceiverNonNull]{
    import edu.colorado.plv.bounder.lifestate.LifeState.LSConstraint
    import edu.colorado.plv.bounder.symbolicexecutor.state.Equals
    import edu.colorado.plv.bounder.symbolicexecutor.state.IntVal
    //val firstRun = "alarm"
    val initial = read[InitialQuery]("""
{
  "methodName": "boolean onMenuItemClick(android.view.MenuItem)",
  "line": 279,
  "t": "ReceiverNonNull",
  "matcher": "android.content.Intent setFlags(int)",
  "className": "org.navitproject.navit.NavitGraphics$NavitView"
}
    """).asInstanceOf[ReceiverNonNull]
    val reachableDual = Reachable(initial.sig, initial.line)
    assert(mConfig.size == 1)
    val config = mConfig.head._2.copy(
        componentFilter = Some(Seq("org.navitproject.navit.NavitGraphics.*")))
    val MenuItem = Set("android.view.MenuItem")
    val m = NamedPureVar("m")
    val m2 = NamedPureVar("m2")
    val l = NamedPureVar("l")
    val v = NamedPureVar("v")
    val i = NamedPureVar("i")
    val inum = NamedPureVar("inum")

    val MenuItem_setOnMenuItemClickListener = AbsMsg(CIExit, 
                                            SubClassMatcher(MenuItem, 
                                                            "android.view.MenuItem setOnMenuItemClickListener\\(android.view.MenuItem\\$OnMenuItemClickListener\\)",
                                                            "MenuItem_setOnMenuItemClickListener"
                                                           ),
                                            TopVal::i::l::Nil)
    val View = Set("android.view.View")
    val View_onCreateContextMenu = AbsMsg(CBEnter, 
                                          SubClassMatcher(
                                              View,
                                              "void onCreateContextMenu\\(android.view.ContextMenu\\)",
                                              "View_onCreateContextMenu"),
                                          TopVal::m2::m::Nil)
    val View_onCreateContextMenu_exit = View_onCreateContextMenu.copy(mt = CBExit, lsVars = TopVal::i::Nil)
                                              
    
    val MenuItem_onMenuItemClick = ViewSpec.onMenuItemClickI.copy(lsVars = TopVal::l::i::Nil)
    val ContextMenu = Set("android.view.ContextMenu")
    val MenuItem_add = AbsMsg(CIExit,
                              SubClassMatcher(
                                  ContextMenu,
                                  "android.view.MenuItem add\\(int,int,int,java.lang.CharSequence\\)",
                                  "MenuItem_add"),
                              i::m::TopVal::inum::Nil) //TODO change intval 2 to inum
    val MenuItem_clickOnce = LSSpec(l::i::Nil, m::m2::inum::Nil, 
                                    And(
                                        And(MenuItem_setOnMenuItemClickListener, MenuItem_add), NS(View_onCreateContextMenu, MenuItem_onMenuItemClick)), MenuItem_onMenuItemClick)
    //val MenuItem = Set("android.view.MenuItem")
    val MenuItem_getItemID = AbsMsg(CIExit,
                                SubClassMatcher(MenuItem,
                                                "int getItemId\\(\\)",
                                                "MenuItem_getItemID"),
                                    inum::i::Nil)
                                                
    val getItemOnceAdded = LSSpec(inum::i::Nil, m::Nil,MenuItem_add , MenuItem_getItemID)
    
    val MenuItem_add_ionly = MenuItem_add.copy(lsVars = i::Nil)
    val addItemOnlyOnce = LSSpec(i::Nil, Nil, Not(MenuItem_add_ionly), MenuItem_add_ionly)

                                              
                                              
    // val MenuItem_clickMustBeSet = LSSpec(l::Nil,m::m2::Nil, MenuItem_setOnMenuItemClickListener, MenuItem_onMenuItemClick)
    val configWithSpec = config.copy(specSet = PickleSpec(Set(/*MenuItem_clickOnce,*/getItemOnceAdded, addItemOnlyOnce),Set(),Set(MenuItem_getItemID, View_onCreateContextMenu_exit, ViewSpec.onMenuItemClickI, MenuItem_setOnMenuItemClickListener, 
                                                                                                View_onCreateContextMenu, MenuItem_add)))
    val result = "timeout" // TODO: may be able to get this one to work at some point later
    // 6:44 + 8:38 + 7:43 + 4:39 + 9:03 + 4:40 + 2:25 + 6:54 + 3:22
    val specificationTime = 
        (6.minutes + 44.seconds +
         8.minutes + 38.seconds +
         7.minutes + 43.seconds +
         4.minutes + 39.seconds +
         9.minutes +  3.seconds +
         4.minutes + 40.seconds +
         2.minutes + 25.seconds +
         6.minutes + 54.seconds +
         3.minutes + 22.seconds).toDuration
}
Specify_7.writeSpec()
Specify_7.configFile

//Uncomment the following two lines to run Historia on this sample from Table 3
// val result = Specify_7.runHistoriaWithSpec()
// println(s"Result Summary: ${result.resultSummary}")

println(s"App Name: ${Specify_7.appT}")

/home/notebooks/SpecGen/org.navitproject.navit/SensitiveDerefFieldCausedFinish/0/config.json
App Name: Navit


defined object Specify_7
res17_2: File = /home/notebooks/SpecGen/org.navitproject.navit/SensitiveDerefFieldCausedFinish/0/config.json

Statistics on specified
=====

In [19]:
object SpecifiedStatistics{
    val allSpecified = Set[Specifyable[_<:InitialQuery]](
        Specify_0, Specify_1, Specify_2, Specify_3, Specify_4, Specify_5, Specify_6, Specify_7)
    val allI = allSpecified.flatMap{s => s.configWithSpec.specSet.getSpecSpace().allI}
    val specifiedResults = allSpecified.groupBy(_.result).map{case (res,v) => (res,v.size)}
    val specifiedByTag = allSpecified.groupBy(_.configWithSpec.tag).map{case (res,v) => (res.heuristicType,v.size)}
    def printMap[F<:Any,V<:Any](map:Map[F,V]):Unit = {
        map.foreach{case (res,count) => println(s"   ${res}:${count}")}
    }
    
   def toTex(table:Table):String = {
      val v = table.v
      val columns = v.head.headers.size
      assert(columns == v.head.values.size, "found header count: ${columns} and value count: ${values.size}.  These should be equal.")
      val hdr = v.head.headers
      //val colSizes = v.foldLeft(hdr){
      //    case (acc,v) => acc.zip(v.map(_.toString)).map{
      //         case (v1,v2) if v1.size > v2.size => v1
      //         case (_,v2) => v2
      //     }
      // }.map{v => v.size +1}
       
      //val cellsWithSize:List[List[String,Int]] = hdr.zip(colSizes)::(v.map{_.values.map{_.toString}.zip(colSizes)}).toList
      hdr.mkString(" & ") + "\\\\\n" + v.map{row => row.values.mkString(" & ")}.mkString(" \\\\\n")
      
   }   
}
println(s"""abstract messages specified: ${SpecifiedStatistics.allI.size}""")
println(s"""results count: """)
SpecifiedStatistics.printMap(SpecifiedStatistics.specifiedResults)
println(s"""specified by tag:""")
SpecifiedStatistics.printMap(SpecifiedStatistics.specifiedByTag)



abstract messages specified: 27
results count: 
   true alarm:1
   proven with spec:3
   timeout:4
specified by tag:
   SensitiveDerefFieldCausedFinish:1
   SensitiveDerefCallinCaused:6
   Disallow.I_CIEnter_Dialogdismiss:1


defined object SpecifiedStatistics

Table 3
-------

In [20]:
val tbl = Table(SpecifiedStatistics.allSpecified.toSeq)
// println(SpecifiedStatistics.toTex(tbl))

/home/notebooks/SpecGen/ch.blinkenlights.android.vanilla/SensitiveDerefCallinCaused/0/config.json
/home/notebooks/SpecGen/de.blinkt.openvpn/SensitiveDerefCallinCaused/0/config.json
/home/notebooks/SpecGen/org.navitproject.navit/SensitiveDerefFieldCausedFinish/0/config.json
/home/notebooks/SpecGen/com.seafile.seadroid2/SensitiveDerefCallinCaused/0/config.json
/home/notebooks/SpecGen/com.nutomic.syncthingandroid/SensitiveDerefCallinCaused/0/config.json
/home/notebooks/SpecGen/org.connectbot/Disallow.I_CIEnter_Dialogdismiss/0/config.json
/home/notebooks/SpecGen/com.darshancomputing.BatteryIndicatorPro/SensitiveDerefCallinCaused/0/config.json
/home/notebooks/SpecGen/de.danoeh.antennapod/SensitiveDerefCallinCaused/0/config.json


app,location,pattern,cb/cbret,ci,filt cb/cbret,filt ci,specs,cb,cb\%,cbret,cbret\%,ci,ci\%,time,res,spec time
Vanilla Music,PreferencesActivity,\apGa,473,5440,35,115,3,14,40,0,0,10,9,,timeout,83
OpenVPN,VPNProfileList,\apGa,938,9628,938,9628,2,49,5,5,1,167,2,0,proven with spec,5
Navit,NavitGraphics,\cbFi,279,2424,13,199,2,2,15,1,8,7,4,,timeout,54
Seafile,SslConfirmDialog,\apGa,2464,19562,2464,19562,2,54,2,9,0,68,0,1,proven with spec,5
Syncthing,SettingsActivity,\apGa,709,5573,7,396,3,2,29,1,14,35,9,,timeout,9
Connectbot,GeneratePubkeyActivity,\ymDi,1476,10211,19,248,1,0,0,0,0,2,1,754,true alarm,4
BatteryBot,PersistentFragment,\apGa,171,2470,171,2470,1,7,4,4,2,55,2,1,proven with spec,3
Antennapod,SynchronizationPreferencesFragment,\apGa,3906,23056,5,87,3,1,20,0,0,23,26,,timeout,35


Make Subset of Samples For Minimal Resources Run
------------------------------------------------

This cell compares the subset with the original run based on results in the `dataSubset` directory.
If you have just imported the database of our results, it won't do anything interesting.
After you have run the subset, it will be comparing your results to our results.

If you would like to change the subset to compare, simply delete the contents of the `dataSubset` directory, re-load the database of our results and run this cell again.  Selection parameters 


In [21]:
//case class JoinedResultRow(tag:String, summary:String, loc:List[String], qry:String, config:String, apk:String) 

// Sample selection criteria
val subsetDir = File("/home/notebooks/dataSubset")
val subsetSampleCount = 10 // randomly choose 20 to reproduce that match the criteria
val subsetCriteria: JoinedResultRow=>Boolean = 
  row => row.summary.contains("Witnessed") || row.summary.contains("Unreachable") // Timeouts and mem errors are uninteresting
val rnd = new Random(42)
(() => {
    def resultFileFromCfgFile(cfg:File):File = {
        val fType = ".json"
        val strRep = cfg.toString
        assert(strRep.endsWith(fType), s"Can only be used on $fType files, got: ${strRep}")
        File(strRep.dropRight(fType.size) + ".result")
    }
    val resultsFromDB = ReadResults.selectResults()
    val getCfgPath = (expDir / "data").glob("**/*.json").map{f => (read[RunConfig](f.contentAsString),f)}.toMap
    val existingConfigs = subsetDir.glob("**/*.json").toList
        .filter{!_.toString.contains(".ipynb_checkpoints")} // ignore jupyter stuff
    println(s"Found ${existingConfigs.size} existing configs.")
    val randomRows = 
        if(existingConfigs.isEmpty){ // if no configs are found, generate a set of configs from the database
            println("No subset found, creating new subset.")
            assert(resultsFromDB.size > 1000, "Please load our pre-run data before generating samples.")
            val chosen = rnd.shuffle(ReadResults.topResultsByQuery                 // randomize 
                                    .filter{subsetCriteria}.toList)                // apply selection criteria
                                    .take(subsetSampleCount)                       // take desired number of samples
                                    .map{a => 
                                        (getCfgPath(read[RunConfig](a.config)),    // find path for config
                                         a.summary,                                // associate summary with config path
                                         "no comparison")}                         // comparison not relevant yet
            chosen.foreach{ case (cfgPath, summary, comparison) => 
                val cfgDestination = File("/home/notebooks/dataSubset") / cfgPath.toString.split("/data/")(1)
                cfgDestination.parent.createDirectories()
                BounderUtil.runCmdStdout(s"cp $cfgPath $cfgDestination") // workaround for weird reflection error
                // cfgPath.copyTo(cfgDestination,true)
                val summaryFileDestination = resultFileFromCfgFile(cfgDestination)
                summaryFileDestination.overwrite(summary)
            }
            chosen
        }else{ 
            existingConfigs.map{ configFile =>  // check each config and result in dataSubset against the current database
                val config = read[RunConfig](configFile.contentAsString)
                val comparisonFromOurRun = resultFileFromCfgFile(configFile).contentAsString
                val matchingRows = resultsFromDB.filter{r => 
                    read[RunConfig](r.config) == config
                }
                if(matchingRows.size == 0){
                    (configFile, "Did not finish in recent run.", comparisonFromOurRun)
                }else if(matchingRows.size == 1){
                    val summaryFromDB = matchingRows.head.summary
                    (configFile, summaryFromDB, comparisonFromOurRun)
                }else {
                    throw new IllegalStateException(s"Found multiple rows for config file: ${configFile} ${matchingRows.size}")
                }
            }
        }
    
    println(s"size of random rows: ${randomRows.size}")
    assert(randomRows.size == subsetSampleCount, "bad sample count")
    
    val consistent = randomRows.map{row => 
        println("---")
        println(s"config file: ${row._1} ")
        println(s"result from most recent run:${row._2}")
        println(s"comparison from paper run: ${row._3}")
        val comparison = row._2 == row._3
        println(s"result is the same: ${row._2 == row._3}")
        comparison
    }.reduce(_&&_)
    println(s"It is ${consistent} that all re-run locations have the same result.")
})()


Found 10 existing configs.
size of random rows: 10
---
config file: /home/notebooks/dataSubset/dev.ukanth.ufirewall/SensitiveDerefCallinCaused_8.json 
result from most recent run:"\"Unreachable\""
comparison from paper run: "\"Unreachable\""
result is the same: true
---
config file: /home/notebooks/dataSubset/net.wigle.wigleandroid/SensitiveDerefCallinCaused_7.json 
result from most recent run:"\"Witnessed\""
comparison from paper run: "\"Witnessed\""
result is the same: true
---
config file: /home/notebooks/dataSubset/be.digitalia.fosdem/SensitiveDerefFieldCausedSynch_6.json 
result from most recent run:"\"Unreachable\""
comparison from paper run: "\"Unreachable\""
result is the same: true
---
config file: /home/notebooks/dataSubset/com.darshancomputing.BatteryIndicatorPro/SensitiveDerefCallinCaused_22.json 
result from most recent run:"\"Witnessed\""
comparison from paper run: "\"Witnessed\""
result is the same: true
---
config file: /home/notebooks/dataSubset/de.syss.MifareClassicTo

subsetDir: File = /home/notebooks/dataSubset
subsetSampleCount: Int = 10
subsetCriteria: JoinedResultRow => Boolean = ammonite.$sess.cmd20$Helper$$Lambda$3427/0x0000000800f9e040@3655b4a2
rnd: Random = scala.util.Random@71b352a